<a href="https://colab.research.google.com/github/saribasmetehan/Training-a-Turkish-NER-Model-with-SpaCy/blob/main/Training_a_Turkish_NER_Model_with_SpaCy_Library_Using_Transformers_Architecture_A_Step_by_Step_Guide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import spacy
import json
import requests
import pandas as pd
import os
from tqdm import tqdm
from spacy.tokens import DocBin

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
spacy.prefer_gpu()

In [ ]:
!pip install https://huggingface.co/turkish-nlp-suite/tr_core_news_trf/resolve/main/tr_core_news_trf-any-py3-none-any.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.2/849.2 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 919.6/919.6 kB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 78.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 26.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Usin

In [ ]:
url = "https://raw.githubusercontent.com/turkish-nlp-suite/Atis_Turkish/main/json/atis_train.json"

response = requests.get(url)

with open("atis_train.json", "wb") as f:
    f.write(response.content)

with open("atis_train.json") as f:
    data = json.load(f)

TRAIN_DATA = []
for item in data:
    text = item["text"]
    entities = [(entity["start"], entity["end"], entity["entity"]) for entity in item["entities"]]
    TRAIN_DATA.append((text, {"entities": entities}))


In [ ]:
TRAIN_DATA

[("Pittsburgh'tan Atlanta'ya 25 Nisan'da gidiş 6 Mayıs'ta dönüşü olan bir gidiş dönüş uçuşunun maliyeti nedir",
  {'entities': [(0, 14, 'fromloc.city_name'),
    (15, 25, 'toloc.city_name'),
    (26, 28, 'depart_date.day_number'),
    (29, 37, 'depart_date.month_name'),
    (44, 45, 'return_date.day_number'),
    (46, 54, 'return_date.month_name')]}),
 ("Şimdi Fort Worth'dan ayrılan ve en geç gelecek pazartesi akşam 2'ye kadar Denver'e varacak bir uçağa ihtiyacım var",
  {'entities': [(6, 20, 'fromloc.city_name'),
    (39, 46, 'arrive_date.date_relative'),
    (47, 56, 'arrive_date.day_name'),
    (57, 62, 'arrive_time.period_of_day'),
    (63, 67, 'arrive_time.time'),
    (74, 82, 'toloc.city_name')]}),
 ("Önümüzdeki çarşamba gidiş ertesi gün dönüşü olan Kansas City'den Chicago'ya giden bir uçuşa ihtiyacım var",
  {'entities': [(0, 10, 'depart_date.date_relative'),
    (11, 19, 'depart_date.day_name'),
    (26, 36, 'return_date.today_relative'),
    (49, 55, 'fromloc.state_name'),
   

In [ ]:
url = "https://raw.githubusercontent.com/turkish-nlp-suite/Atis_Turkish/main/json/atis_test.json"

response = requests.get(url)

with open("atis_train.json", "wb") as f:
    f.write(response.content)

with open("atis_train.json") as f:
    data_test = json.load(f)

TEST_DATA = []
for item in data_test:
    text = item["text"]
    entities = [(entity["start"], entity["end"], entity["entity"]) for entity in item["entities"]]
    TEST_DATA.append((text, {"entities": entities}))

In [ ]:
TEST_DATA

[("Dallas ve Baltimore arasında on Ağustos'ta kalkıp on iki Ağustos'ta geri dönen ucuz uçuşlar hangileri",
  {'entities': [(0, 6, 'fromloc.city_name'),
    (10, 19, 'toloc.city_name'),
    (29, 31, 'depart_date.day_number'),
    (32, 42, 'depart_date.month_name'),
    (50, 56, 'return_date.day_number'),
    (57, 67, 'return_date.month_name')]}),
 ("Nashville'den Seattle'ye giden saat 3'ten önce varışlı bir uçuş istiyorum",
  {'entities': [(0, 13, 'fromloc.city_name'),
    (14, 24, 'toloc.city_name'),
    (36, 41, 'depart_time.time'),
    (42, 46, 'depart_time.time_relative')]}),
 ("Kansas'tan Chicago'ya önümüzdeki Çarşamba kalkan ve ertesi gün dönen bir uçuş istiyorum",
  {'entities': [(0, 10, 'fromloc.state_name'),
    (11, 21, 'toloc.city_name'),
    (22, 32, 'depart_date.date_relative'),
    (33, 41, 'depart_date.day_name'),
    (52, 62, 'return_date.today_relative')]}),
 ('Yemek kurallarını açıklayın', {'entities': [(0, 5, 'meal')]}),
 ("Atlanta'dan San Francisco'ya yarından sonrak

In [ ]:
nlp = spacy.blank("tr")
nlp = spacy.load("tr_core_news_trf")

db = DocBin()

for text, annot in tqdm(TRAIN_DATA):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="strict")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)

os.chdir(r'/content/drive/MyDrive/spacy_medium')
db.to_disk("/content/drive/MyDrive/spacy_medium/train.spacy")

100%|██████████| 4274/4274 [00:00<00:00, 7401.21it/s]


In [ ]:
db_test = DocBin()

for text, annot in tqdm(TEST_DATA):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="strict")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents
    db_test.add(doc)

os.chdir(r'/content/drive/MyDrive/spacy_medium')
db.to_disk("/content/drive/MyDrive/spacy_medium/test.spacy")

100%|██████████| 586/586 [00:00<00:00, 8073.36it/s]


In [ ]:
%cd /content/drive/MyDrive/spacy_medium
!python -m spacy init fill-config base_config.cfg config.cfg

/content
2024-05-29 18:37:40.434765: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-29 18:37:40.434825: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-29 18:37:40.436288: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-29 18:37:41.474914: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train /content/drive/MyDrive/spacy_medium/config.cfg --output ./output --paths.train /content/drive/MyDrive/spacy_medium/train.spacy --paths.dev /content/drive/MyDrive/spacy_medium/test.spacy --gpu-id 0

2024-05-29 18:39:17.583445: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-29 18:39:17.583501: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-29 18:39:17.585286: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-29 18:39:18.880481: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2024-05-29 18:39:21,565] [INFO] Set up nlp object from config
[2024-05-29 

In [ ]:
nlp = spacy.load("/content/drive/MyDrive/spacy_medium/output/model-best")
doc = nlp("Malatya'dan İstanbul'a 20.45'te uçuş var mı?")

spacy.displacy.render(doc, style="ent", jupyter=True)